In [316]:
import os #Files in listen abgreifen
import pandas as pd
import camelot # um Tabellen im PDFs zu erkennen
import PyPDF2 # um Text aus PDFs zu lesen
import re

In [317]:
regex_gnummer = r"(\d\d\.\S+)"
regex_datum = r"\d\d\.\d\d\.\d\d\d\d \d\d\:\d\d\:\d\d"

In [318]:
files = os.listdir("Sessionsprotokolle")

In [319]:
files

['SR_5101_1912.pdf',
 'SR_5117_2212.pdf',
 'SR_5105_2009.pdf',
 'SR_5110_2106.pdf',
 'SR_5107_2012.pdf',
 'SR_5119_2304.pdf',
 'SR_5112_2112.pdf',
 'SR_5103_2005.pdf',
 'SR_5118_2303.pdf',
 'SR_5115_2206.pdf',
 'SR_5102_2003.pdf',
 'SR_5111_2109.pdf',
 'SR_5116_2209.pdf',
 'SR_5113_2203.pdf',
 'SR_5104_2006.pdf',
 'SR_5121_2306.pdf',
 'SR_5108_2103.pdf']

In [320]:
master_df_list = []

for file in files:

    print(file)
    
    session = file
    
    dokument = PyPDF2.PdfReader("Sessionsprotokolle/" + str(file))
    
    seitenliste = []

    laenge = len(dokument.pages)

    for seite in range(laenge):
        inhalt = dokument.pages[seite].extract_text()
        if inhalt[:8] == "Geschäft":
            nummer = seite
            seitenliste.append(nummer)

    for seite in seitenliste:
        inhalt = dokument.pages[seite].extract_text()
        datenliste = inhalt.split("\n")
    
        for zeile in range(len(datenliste)):
            if "Abstimmung vom" in datenliste[zeile]:
        
                datum = re.findall(regex_datum, datenliste[zeile])[0]
            
                print(datum)
    
        for zeile in range(len(datenliste)):
            if "Geschäft / Objet:" in datenliste[zeile]:
                geschäftszeile = datenliste[(zeile+1)]
            
                gnummer = (re.findall(regex_gnummer, geschäftszeile))[0]
                geschäft = geschäftszeile.strip(str(gnummer)+" ")
                
                print(gnummer)
                print(geschäft)
                print(" ")
                
        for zeile in range(len(datenliste)):
            if "Gegenstand" in datenliste[zeile]:
                gegenstand = str(datenliste[zeile]).replace("Gegenstand / Objet du vote: ","")
            
                
                
        for zeile in range(len(datenliste)):
            if "+ Ja / oui / si " in datenliste[zeile]:
                jastimmen = int(str(datenliste[zeile]).replace("+ Ja / oui / si ",""))

        for zeile in range(len(datenliste)):    
            if "- Nein / non / no " in datenliste[zeile]:
                neinstimmen = int(str(datenliste[zeile]).replace("- Nein / non / no ",""))

        if jastimmen > neinstimmen: 
            resultat ="+"

        if jastimmen < neinstimmen: 
            resultat ="-"
            
        if abs(jastimmen - neinstimmen) <= 2:
            knapp = "Ja"
            
        if abs(jastimmen - neinstimmen) > 2:
            knapp = "Nein"
      
        # So holen wir die richtigen 24 Elemente fürs Abstimmungsverhalten aus der Tabelle
        try:
            tabellenanfang = datenliste.index(' ')+1
        
        #das ist der Ausnahmefall: So suchen wir die Abstimmungszeile und nehmen die nächste
        except:
            def first_substring(strings, substring):
                return next(i for i, string in enumerate(strings) if substring in string)

            tabellenanfang = first_substring(datenliste, 'Abstimmung vom / Vote du:')
        
        tabelle = datenliste[tabellenanfang:(tabellenanfang+24)]
        
        # So flicken wir die Gmür-Schnönenberger:
        tabelle.remove("Gmür-")
        tabelle = [w.replace('SchönenbergerAndrea', 'Gmür-Schönenberger Andrea') for w in tabelle]
        
        # und jetzt trennen wir jede Zeile beim ersten Auftauchen eines Kantons (jenes mit Leerschlag)
        tabelle_getrennt = []
        kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

        for paar in tabelle:
            for kanton in kantonsliste:
                trenner = kanton + " "
                if trenner in paar:
                    trennung = paar.split(trenner)
            
                    # (dann müssen wir den bei wplit verloren gegangenen Kanton, den Trenner, wieder hinzufügen)
                    trennung[0] = trennung[0]+kanton
            
                    tabelle_getrennt.append(trennung[0])
                    tabelle_getrennt.append(trennung[1])
                    
        for eintrag in tabelle_getrennt:
            eintrag_neu = eintrag.rsplit(" ", 2)
            
            name = eintrag_neu[0]
            kanton = eintrag_neu[2]
            verhalten = eintrag_neu[1]
            
            mini_dict = {
            "Session" : session,
            "Datum" : datum,
            "Nummer" : gnummer,
            "Geschäft" : geschäft,
            "Gegenstand" : gegenstand,
            "Resultat" : resultat,
            "knapp?" : knapp,
            "Politiker" : name,
            "Kanton" : kanton,
            "Verhalten" : verhalten}
    
            master_df_list.append(mini_dict)

SR_5101_1912.pdf
16.12.2019 18:29:16
13.094
OR. Schutz bei Meldung von Unregelmässigkeiten am Arbeitsplatz
 
18.12.2019 11:47:01
16.077
OR. Aktienrecht
 
02.12.2019 18:34:49
16.452
Pa.Iv. Rösti. Ausbau der Wasserkraft zur Stromerzeugung und Stromspeicherung. Anpassung der
 
20.12.2019 08:25:41
16.452
Pa.Iv. Rösti. Ausbau der Wasserkraft zur Stromerzeugung und Stromspeicherung. Anpassung der
 
18.12.2019 13:17:34
17.059
Datenschutzgesetz. Totalrevision und Änderung weiterer Erlasse zum Datenschutz
 
02.12.2019 19:42:36
17.405
Pa.Iv. Burkart. Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene
 
20.12.2019 08:26:18
17.405
Pa.Iv. Burkart. Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene
 
05.12.2019 09:12:54
18.057-1
Bundesgesetz über die Enteignung. Änderung
 
20.12.2019 08:26:54
18.068
Asylgesetz. Änderung (Rahmenkredit Migration; 2. Schweizer Beitrag an ausgewählte EU-Staaten)
 
20.12.2019 08:27:24
18.085
Bevö

13.12.2022 10:28:28
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:29:13
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:29:47
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:30:23
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:31:07
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:31:50
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:32:59
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:33:35
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:40:58
20.022-1
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:42:37
20.022-3
Agrarpolitik ab 2022 (AP22+)
 
13.12.2022 10:43:10
20.022-3
Agrarpolitik ab 2022 (AP22+)
 
15.12.2022 08:56:15
20.034-1
Bundesgesetz über das Internationale Privatrecht. Änderung
 
15.12.2022 09:01:04
20.034-1
Bundesgesetz über das Internationale Privatrecht. Änderung
 
15.12.2022 09:09:21
20.034-1
Bundesgesetz über das Internationale Privatrecht. Änderung
 
15.12.2022 09:11:16
20.034-1
Bundesgesetz über das Internationale Priv

29.11.2022 10:18:44
22.046-1
Covid-19-Gesetz. Änderung (Verlängerung und Änderung ausgewählter Bestimmungen)
 
29.11.2022 10:20:43
22.046
Covid-19-Gesetz. Änderung (Verlängerung und Änderung ausgewählter Bestimmungen)
 
06.12.2022 12:42:35
22.046-1
Covid-19-Gesetz. Änderung (Verlängerung und Änderung ausgewählter Bestimmungen)
 
14.12.2022 15:09:30
22.046-1
Covid-19-Gesetz. Änderung (Verlängerung und Änderung ausgewählter Bestimmungen)
 
16.12.2022 08:36:35
22.046-1
Covid-19-Gesetz. Änderung (Verlängerung und Änderung ausgewählter Bestimmungen)
 
29.11.2022 08:31:13
22.047-1
Gültigkeitsdauer der dringlichen Bestimmung zum Covid-19-Test bei der Ausschaffung (Art. 72 AIG)
 
29.11.2022 08:32:20
22.047-1
Gültigkeitsdauer der dringlichen Bestimmung zum Covid-19-Test bei der Ausschaffung (Art. 72 AIG)
 
08.12.2022 08:19:48
22.047-1
Gültigkeitsdauer der dringlichen Bestimmung zum Covid-19-Test bei der Ausschaffung (Art. 72 AIG)
 
16.12.2022 08:37:05
22.047-1
Gültigkeitsdauer der dringlichen B

22.09.2020 10:23:28
19.079-1
Programme "SUPERB" und "ERP Systeme V/ar". Verpflichtungskredite
 
22.09.2020 10:25:58
19.079-1
Programme "SUPERB" und "ERP Systeme V/ar". Verpflichtungskredite
 
14.09.2020 19:11:27
19.475-1
Pa. Iv. WAK-SR. Das Risiko beim Einsatz von Pestiziden reduzieren
 
08.09.2020 10:54:29
20.016-1
Obligatorisches Referendum für völkerrechtliche Verträge mit Verfassungscharakter. Änderung von Artikel
 
10.09.2020 11:57:11
20.023-1
IWF. Beitritt zu den geänderten Neuen Kreditvereinbarungen
 
25.09.2020 08:43:44
20.027-1
Weiterentwicklung des Schengen-Besitzstands. Europäisches Reiseinformations- und -genehmigungssystem
 
25.09.2020 08:44:19
20.027-2
Weiterentwicklung des Schengen-Besitzstands. Europäisches Reiseinformations- und -genehmigungssystem
 
22.09.2020 08:38:29
20.028-2
Förderung von Bildung, Forschung und Innovation in den Jahren 2021-2024
 
25.09.2020 08:44:49
20.028-12
Förderung von Bildung, Forschung und Innovation in den Jahren 2021-2024
 
25.09.2020 08:4

18.06.2021 08:19:41
18.079-1
Für eine starke Pflege (Pflegeinitiative). Volksinitiative
 
18.06.2021 08:20:17
18.458-1
Pa. Iv. Rieder. Differenzbereinigungsverfahren bei Motionen
 
31.05.2021 18:25:10
19.043-1
Bekämpfung des missbräuchlichen Konkurses. Bundesgesetz
 
18.06.2021 08:20:58
19.046-2
Bundesgesetz über die Krankenversicherung. Änderung (Massnahmen zur Kostendämpfung - Paket 1)
 
18.06.2021 08:21:37
19.083-1
Ja zum Tier- und Menschenversuchsverbot - Ja zu Forschungswegen mit Impulsen für Sicherheit un
 
03.06.2021 12:25:36
19.085-1
Embargogesetz. Änderung
 
18.06.2021 08:22:11
19.400-1
Pa. Iv. SPK-SR. Mehr Transparenz bei der Politikfinanzierung
 
03.06.2021 11:58:11
20.022-4
Agrarpolitik ab 2022 (AP22+)
 
16.06.2021 13:09:18
20.026-1
Zivilprozessordnung. Änderung
 
07.06.2021 18:25:39
20.030-2
Förderung der Kultur in den Jahren 2021-2024
 
01.06.2021 10:23:25
20.038-1
Massnahmenpaket zugunsten der Medien
 
18.06.2021 08:22:48
20.038-1
Massnahmenpaket zugunsten der Medien
 
1

...

16.03.2022 08:53:00
21.078-1
Internationale Währungshilfe. Weiterführung
 
16.03.2022 08:54:02
21.078-1
Internationale Währungshilfe. Weiterführung
 
01.03.2022 11:50:21
21.081-1
Assistenzdienst der Armee zur Unterstützung der zivilen Behörden im Rahmen der Massnahmen zur
 
16.03.2022 11:36:38
21.306
Kt. Iv. Jura. Internetgiganten sind zu besteuern!
 
16.03.2022 11:56:19
21.316
Kt. Iv. Genf. Für eine Verlängerung der Frist bei Zahlungsrückständen der Mieterin oder des Mieters
 
17.03.2022 12:48:08
21.3004
Mo. WAK-SR. Anpassung der Suisse-Bilanz und deren Grundlagen an die effektiven Verhältnisse
 
15.03.2022 12:00:43
21.3282
Mo. Jositsch. Wiedereinführung des Botschaftsasyls
 
15.03.2022 12:32:06
21.3598
Mo. WAK-NR. Änderung des Bundesgesetzes über den Erwerb von Grundstücken durch Personen im
 
16.03.2022 09:49:20
21.3928
Mo. Pfister Gerhard. Schweizerische Unfallversicherungsanstalt. Prüfung durch die Eidgenössische
 
17.03.2022 12:38:53
21.4188
Mo. Wicki. Homeoffice. Gelebte und akz

10.06.2020 13:07:33
19.401-2
Pa. Iv. SGK-NR. Für eine Stärkung der Pflege, für mehr Patientensicherheit und mehr Pflegequalität
 
10.06.2020 13:08:27
19.401-3
Pa. Iv. SGK-NR. Für eine Stärkung der Pflege, für mehr Patientensicherheit und mehr Pflegequalität
 
10.06.2020 13:09:22
19.401-3
Pa. Iv. SGK-NR. Für eine Stärkung der Pflege, für mehr Patientensicherheit und mehr Pflegequalität
 
10.06.2020 13:10:16
19.401-4
Pa. Iv. SGK-NR. Für eine Stärkung der Pflege, für mehr Patientensicherheit und mehr Pflegequalität
 
19.06.2020 08:42:37
19.471-1
Pa. Iv. Comte. Opfer fürsorgerischer Zwangsmassnahmen. Fristverlängerung
 
19.06.2020 08:43:28
20.008-1
Aussenwirtschaftspolitik 2019. Bericht
 
19.06.2020 08:44:13
20.008-2
Aussenwirtschaftspolitik 2019. Bericht
 
19.06.2020 08:44:56
20.008-3
Aussenwirtschaftspolitik 2019. Bericht
 
08.06.2020 17:06:35
20.024-1
Weltbankgruppe und Afrikanische Entwicklungsbank. Kapitalerhöhung
 
08.06.2020 17:07:27
20.024-1
Weltbankgruppe und Afrikanische Entwickl

06.06.2023 08:57:55
21.019
Mehrwertsteuergesetz. Teilrevision
 
06.06.2023 09:10:56
21.019
Mehrwertsteuergesetz. Teilrevision
 
16.06.2023 08:21:45
21.019-1
Mehrwertsteuergesetz. Teilrevision
 
01.06.2023 12:35:59
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 12:51:34
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 15:09:34
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 15:37:09
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 16:03:16
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 16:43:38
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 17:06:45
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
01.06.2023 17:30:01
21.047-1
Sichere Stromversorgung mit erneuerbaren Energien. Bundesgesetz
 
08.06.2023 08:28:31
21.047-1
Sichere Stromvers

14.06.2023 10:07:16
23.028-6
Standortförderung 2024-2027
 
14.06.2023 10:17:02
23.028-6
Standortförderung 2024-2027
 
14.06.2023 10:17:41
23.028-6
Standortförderung 2024-2027
 
14.06.2023 10:18:15
23.028-6
Standortförderung 2024-2027
 
14.06.2023 10:19:05
23.028-7
Standortförderung 2024
 
14.06.2023 10:19:35
23.028-7
Standortförderung 2024
 
07.06.2023 11:50:42
23.402-0
Pa. Iv. SiK-SR. Änderung des Kriegsmaterialgesetzes
 
31.05.2023 11:23:48
23.404
Pa. Iv. Bü-SR. Teuerungsausgleich für die Einkommen und Entschädigungen der Ratsmitglieder
 
08.06.2023 11:16:26
23.427
Pa. Iv. Bü-NR. Einsetzung einer PUK zur Untersuchung der Verantwortlichkeiten der Behörden und Organe
 
08.06.2023 11:21:06
23.427
Pa. Iv. Bü-NR. Einsetzung einer PUK zur Untersuchung der Verantwortlichkeiten der Behörden und Organe
 
06.06.2023 10:22:49
23.3014
Mo. SGK-SR. Nationaler Krebsplan
 
06.06.2023 10:48:11
23.3018
Mo. WBK-SR. Problematischen Einsatz von Nutri-Score unterbinden
 
05.06.2023 18:52:09
23.3031
Mo. Mü

19.03.2021 08:34:22
21.008-1
Aussenwirtschaftspolitik 2020. Bericht
 
19.03.2021 08:34:56
21.008-2
Aussenwirtschaftspolitik 2020. Bericht
 
04.03.2021 10:17:35
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 11:23:55
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 11:24:41
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 12:03:09
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 13:22:11
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 13:23:00
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 13:25:07
21.016-2
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 13:24:12
21.016-2
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 13:26:16
21.016-3
Covid-19-Gesetz. Änderung und Zusatzkredit
 
04.03.2021 13:26:57
21.016-3
Covid-19-Gesetz. Änderung und Zusatzkredit
 
10.03.2021 09:05:26
21.016-1
Covid-19-Gesetz. Änderung und Zusatzkredit
 
10.03.2021 09:49:47
21.016-1
Covid-19-Gesetz. 

In [321]:
master_df = pd.DataFrame(master_df_list)
master_df

,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,knapp?,Politiker,Kanton,Verhalten
0,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Bauer Philippe,NE,+
1,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Kuprecht Alex,SZ,-
2,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Baume-Schneider Elisabeth,JU,-
3,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Levrat Christian,FR,=
4,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Bischof Pirmin,SO,+
...,...,...,...,...,...,...,...,...,...,...
68573,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Nein,Zanetti Roberto,SO,+
68574,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Nein,Juillard Charles,JU,+
68575,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Nein,Z'graggen Heidi,UR,+
68576,SR_5108_2103.pdf,19.03.2021 08:36:11,21.016-3,Covid-19-Gesetz. Änderung und Zusatzkredit,Gesamtabstimmung,+,Nein,Knecht Hansjörg,AG,+


In [322]:
master_df["Datum"].value_counts().to_frame().tail(5)

,Datum
02.12.2021 12:40:17,46
13.06.2023 08:39:21,44
22.09.2022 15:04:05,44
26.09.2022 17:13:34,44
13.06.2022 16:38:17,44


## Loop, der die Standesstimme auswertet

In [323]:
# Das hier generiert uns eine Liste mit allen Geschäften und allen Kantonen:
geschaeftsliste = master_df["Datum"].value_counts().index.to_list()
kantonsliste = ["VD", "TG", "TI", "AG", "JU", "GL", "UR", "SO", "NE", "SZ", "GR", "BE", "SG", "ZH", "LU", "SH", "ZG", "GE", "VS", "FR", "OW", "BL", "AI", "BS", "AR", "NW"]

In [324]:
# So schalte ich die Fehlermeldung aus
pd.options.mode.chained_assignment = None  # default='warn'

In [325]:
df_komplett = pd.DataFrame()

for geschaeft in geschaeftsliste:
    geschaeft_df = master_df[master_df["Datum"] == geschaeft]
    
    for kanton in kantonsliste:
        kanton_df = geschaeft_df[geschaeft_df["Kanton"] == kanton]
        standesstimme = kanton_df["Verhalten"].value_counts().to_dict()
        position_gleiche = kanton_df.index.to_list()
        
        for zeile in position_gleiche:
    
            #hier erledigen wir jene Kantone, die nur einen EIntrag haben
            if len(kanton_df) <= 1:
                kanton_df.loc[zeile, "Standesstimme"] = "Halbkanton"
     
            #hier erledigen wir jene, die aus verschiedenen Gründen nicht vollzählig waren
            elif "E" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
            elif "P" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
            elif "0" in standesstimme:
                kanton_df.loc[zeile, "Standesstimme"] = "nicht vollzählig"
   
            #hier kommen jene, die gelich abgstimmt haben
            elif "+" in standesstimme:
                if standesstimme["+"] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide dafür"
                if standesstimme["+"] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
            elif "-" in standesstimme:
                if standesstimme["-"] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide dagegen"
                if standesstimme["-"] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
            elif "=" in standesstimme:
                if standesstimme["="] == 2:
                    kanton_df.loc[zeile, "Standesstimme"] = "beide enthalten"
                if standesstimme["="] == 1:
                    kanton_df.loc[zeile, "Standesstimme"] = "uneinig"
        
            #und schliesslich die anderen
            else:
                kanton_df.loc[zeile, "Standesstimme"] = "leer"
                
            if len(position_gleiche) > 1:
                if kanton_df.loc[position_gleiche[0], "Verhalten"] == "0" and kanton_df.loc[position_gleiche[1], "Verhalten"] == "0":
                    kanton_df.loc[position_gleiche[0], "Standesstimme"] = "beide absent"
                    kanton_df.loc[position_gleiche[1], "Standesstimme"] = "beide absent"
                
            for index, row in kanton_df.iterrows():
                if row["Verhalten"] == row["Resultat"]:
                    kanton_df.loc[index,"siegreich"] = "Ja"
                elif row["Verhalten"] == "0" or row["Verhalten"] == "P" or row["Verhalten"] == "E":
                    kanton_df.loc[index,"siegreich"] = "---"
                else:
                    kanton_df.loc[index,"siegreich"] = "Nein"
                    
        df_komplett = df_komplett.append(kanton_df)

In [326]:
df_komplett.head(50)

,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,knapp?,Politiker,Kanton,Verhalten,Standesstimme,siegreich
22,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Français Olivier,VD,+,uneinig,Ja
33,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Thorens Goumaz Adèle,VD,-,uneinig,Nein
29,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Stark Jakob,TG,-,uneinig,Nein
32,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Häberli-Koller Brigitte,TG,+,uneinig,Ja
8,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Carobbio Guscetti Marina,TI,-,beide dagegen,Nein
12,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Chiesa Marco,TI,-,beide dagegen,Nein
6,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Burkart Thierry,AG,+,uneinig,Ja
44,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Knecht Hansjörg,AG,-,uneinig,Nein
2,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Baume-Schneider Elisabeth,JU,-,uneinig,Nein
42,SR_5101_1912.pdf,16.12.2019 18:29:16,13.094,OR. Schutz bei Meldung von Unregelmässigkeiten...,Gesamtabstimmung,+,Nein,Juillard Charles,JU,+,uneinig,Ja


In [327]:
len(df_komplett)

68346

In [328]:
df_komplett.to_excel("Datensatz.xlsx")  

In [329]:
df_komplett.to_csv("Datensatz.csv")  

In [332]:
df_komplett[df_komplett["Politiker"] == "Noser Ruedi"]["Standesstimme"].value_counts()

beide dafür         598
nicht vollzählig    533
uneinig             259
beide absent         59
beide dagegen        42
Name: Standesstimme, dtype: int64

In [333]:
df_komplett[df_komplett["Verhalten"] == "="]["Politiker"].value_counts()

Germann Hannes                 85
Salzmann Werner                75
Knecht Hansjörg                73
Stark Jakob                    69
Minder Thomas                  64
Juillard Charles               62
Hegglin Peter                  57
Kuprecht Alex                  56
Chiesa Marco                   50
Caroni Andrea                  48
Mazzone Lisa                   46
Hefti Thomas                   45
Rieder Beat                    44
Fässler Daniel                 44
Stöckli Hans                   40
Bauer Philippe                 39
Vara Céline                    38
Gapany Johanna                 35
Graf Maya                      34
Maret Marianne                 33
Bischof Pirmin                 32
Chassot Isabelle               32
Zopfi Mathias                  31
Français Olivier               29
Engler Stefan                  25
Sommaruga Carlo                24
Thorens Goumaz Adèle           23
Herzog Eva                     23
Zanetti Roberto                22
Dittli Josef  

In [402]:
df_komplett[df_komplett["siegreich"] == "Ja"]["Politiker"].value_counts().mean()

1055.6326530612246

In [338]:
df_komplett[df_komplett["siegreich"] == "Ja"][df_komplett["Standesstimme"] == "uneinig"]["Politiker"].value_counts()

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Bauer Philippe                 306
Stöckli Hans                   295
Burkart Thierry                272
Carobbio Guscetti Marina       244
Bischof Pirmin                 239
Salzmann Werner                229
Juillard Charles               214
Français Olivier               213
Chiesa Marco                   202
Würth Benedikt                 193
Häberli-Koller Brigitte        182
Noser Ruedi                    165
Vara Céline                    162
Hefti Thomas                   160
Reichmuth Othmar               159
Germann Hannes                 133
Gapany Johanna                 131
Michel Matthias                128
Minder Thomas                  109
Zopfi Mathias                   96
Maret Marianne                  93
Z'graggen Heidi                 90
Jositsch Daniel                 90
Engler Stefan                   87
Gmür-Schönenberger Andrea       87
Thorens Goumaz Adèle            84
Dittli Josef                    80
Schmid Martin                   74
Rechsteiner Paul    

In [343]:
df_komplett[df_komplett["siegreich"] == "Ja"][df_komplett["Standesstimme"].isin (["beide dafür", "beide dagegen"])]["Kanton"].value_counts()

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


LU    2198
VS    2184
UR    2160
FR    2096
ZG    2094
GR    2062
SO    1880
VD    1870
GE    1858
AG    1844
JU    1840
SH    1836
NE    1602
SG    1580
SZ    1410
TG    1332
TI    1324
GL    1312
ZH    1252
BE    1118
Name: Kanton, dtype: int64

In [406]:
df_komplett[df_komplett["Verhalten"].isin(["0", "E"])]["Politiker"].value_counts()

Noser Ruedi                    423
Kuprecht Alex                  258
Jositsch Daniel                257
Rechsteiner Paul               225
Stark Jakob                    214
Chiesa Marco                   202
Schmid Martin                  201
Stöckli Hans                   198
Caroni Andrea                  184
Herzog Eva                     179
Mazzone Lisa                   176
Vara Céline                    170
Bischof Pirmin                 166
Reichmuth Othmar               151
Würth Benedikt                 139
Baume-Schneider Elisabeth      120
Z'graggen Heidi                119
Müller Damian                  119
Michel Matthias                113
Gapany Johanna                 112
Ettlin Erich                   111
Dittli Josef                   111
Maret Marianne                 108
Burkart Thierry                107
Français Olivier               105
Thorens Goumaz Adèle           103
Wicki Hans                     102
Gmür-Schönenberger Andrea      102
Hegglin Peter       

In [350]:
df_komplett[df_komplett["Verhalten"].isin(["0", "E"])][df_komplett["Standesstimme"] != "Halbkanton"]["Kanton"].value_counts()

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


ZH    680
SZ    409
SG    365
GR    280
BE    261
TG    253
GE    241
UR    230
SO    224
LU    221
TI    213
ZG    209
VD    208
NE    190
FR    189
JU    188
VS    178
AG    171
GL    142
SH    128
Name: Kanton, dtype: int64

In [351]:
df_komplett[df_komplett["Standesstimme"] == "beide absent"]["Kanton"].value_counts()

ZH    118
GR     58
UR     42
SO     34
SG     34
LU     30
SZ     28
FR     22
GE     20
VS     20
AG     18
ZG     18
BE     16
SH     14
NE     12
TI     12
TG      8
JU      6
VD      4
GL      2
Name: Kanton, dtype: int64

In [356]:
df_komplett[df_komplett["Politiker"] == "Jositsch Daniel"][df_komplett["Verhalten"].isin(["0", "E"])]["Session"].value_counts()

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


SR_5118_2303.pdf    57
SR_5115_2206.pdf    43
SR_5113_2203.pdf    42
SR_5117_2212.pdf    38
SR_5116_2209.pdf    22
SR_5121_2306.pdf    18
SR_5111_2109.pdf     7
SR_5104_2006.pdf     6
SR_5110_2106.pdf     6
SR_5108_2103.pdf     5
SR_5105_2009.pdf     5
SR_5112_2112.pdf     5
SR_5107_2012.pdf     3
Name: Session, dtype: int64

In [397]:
df_komplett[df_komplett["Politiker"] == "Jositsch Daniel"][df_komplett["Standesstimme"] == "beide absent"][df_komplett["knapp?"] == "Ja"]

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,knapp?,Politiker,Kanton,Verhalten,Standesstimme,siegreich
45210,SR_5116_2209.pdf,13.09.2022 10:42:03,21.4341,"Mo. WBK-NR. Kinderwunsch erfüllen, Eizellenspe...",Abstimmung über die Motion,+,Ja,Jositsch Daniel,ZH,0,beide absent,---
30080,SR_5115_2206.pdf,16.06.2022 15:56:53,18.077-1,Raumplanungsgesetz. Teilrevision. 2. Etappe,Art. 18bis Abs. 2,-,Ja,Jositsch Daniel,ZH,0,beide absent,---


In [400]:
df_komplett[df_komplett["Kanton"] == "ZH"][df_komplett["Verhalten"].isin (["0", "E"])][df_komplett["knapp?"] == "Ja"]

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,knapp?,Politiker,Kanton,Verhalten,Standesstimme,siegreich
45183,SR_5116_2209.pdf,13.09.2022 10:42:03,21.4341,"Mo. WBK-NR. Kinderwunsch erfüllen, Eizellenspe...",Abstimmung über die Motion,+,Ja,Noser Ruedi,ZH,0,beide absent,---
45210,SR_5116_2209.pdf,13.09.2022 10:42:03,21.4341,"Mo. WBK-NR. Kinderwunsch erfüllen, Eizellenspe...",Abstimmung über die Motion,+,Ja,Jositsch Daniel,ZH,0,beide absent,---
64942,SR_5121_2306.pdf,14.06.2023 10:51:44,23.3214,Mo. Z'graggen. 175 Jahre Bundesverfassung - Bl...,Abstimmung über die Motion,-,Ja,Jositsch Daniel,ZH,0,nicht vollzählig,---
59928,SR_5121_2306.pdf,01.06.2023 17:06:45,21.047-1,Sichere Stromversorgung mit erneuerbaren Energ...,"Art. 8a Abs. 1, 2, 3, 4, 5, 5bis und 6",+,Ja,Jositsch Daniel,ZH,0,nicht vollzählig,---
59238,SR_5121_2306.pdf,15.06.2023 15:34:55,20.3021-0,Mo. Haab. Importverbot für tierquälerisch erze...,Abstimmung über die Motion,+,Ja,Jositsch Daniel,ZH,0,nicht vollzählig,---
9627,SR_5117_2212.pdf,14.12.2022 16:04:16,22.3392,Mo. SPK-NR. Erweiterte Härtefallregelung zum Z...,Abstimmung über die Motion,-,Ja,Noser Ruedi,ZH,0,nicht vollzählig,---
9673,SR_5117_2212.pdf,14.12.2022 16:15:37,22.3392,Mo. SPK-NR. Erweiterte Härtefallregelung zum Z...,Abstimmung über die Motion,-,Ja,Noser Ruedi,ZH,0,nicht vollzählig,---
5305,SR_5117_2212.pdf,12.12.2022 16:40:33,21.043-1,Bundesgesetz über die Regulierung der Versiche...,Art. 19b Abs. 1 Bst. e,+,Ja,Noser Ruedi,ZH,0,nicht vollzählig,---
28765,SR_5115_2206.pdf,16.06.2022 17:24:21,11.3285,Mo. Fraktion CEG. Erleichterung der Nutzung un...,Abstimmung über die Motion,+,Ja,Noser Ruedi,ZH,0,nicht vollzählig,---
28240,SR_5118_2303.pdf,16.03.2023 09:27:49,22.4274,Mo. APK-SR. Unterstützung der iranischen Zivil...,Abstimmung über die Motion,-,Ja,Jositsch Daniel,ZH,E,nicht vollzählig,---


In [360]:
df_züri = df_komplett[df_komplett["Politiker"] == "Noser Ruedi"][df_komplett["Standesstimme"] == "beide absent"]
df_züri

/Users/bigdata/.virtualenvs/zweitesVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Session,Datum,Nummer,Geschäft,Gegenstand,Resultat,knapp?,Politiker,Kanton,Verhalten,Standesstimme,siegreich
45321,SR_5116_2209.pdf,20.09.2022 08:51:37,22.018-1,Bundesgesetz über die Familienzulagen in der L...,Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---
45275,SR_5116_2209.pdf,20.09.2022 08:51:03,22.018-1,Bundesgesetz über die Familienzulagen in der L...,Art. 25a,+,Nein,Noser Ruedi,ZH,0,beide absent,---
45183,SR_5116_2209.pdf,13.09.2022 10:42:03,21.4341,"Mo. WBK-NR. Kinderwunsch erfüllen, Eizellenspe...",Abstimmung über die Motion,+,Ja,Noser Ruedi,ZH,0,beide absent,---
49367,SR_5113_2203.pdf,07.03.2022 15:38:18,21.036-1,Verordnung über das System FADO. Übernahme und...,Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---
48355,SR_5113_2203.pdf,07.03.2022 15:22:57,16.432-1,Pa. Iv. Graf-Litscher. Gebührenregelung. Öffen...,Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---
36859,SR_5111_2109.pdf,28.09.2021 12:05:14,17.304-1,Kt. Iv. Tessin. Sicherere Strassen jetzt!,Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---
36767,SR_5111_2109.pdf,22.09.2021 10:33:18,16.461-1,"Pa. Iv. Nidegger. EMRK, Strafregister, Restitu...",Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---
35205,SR_5115_2206.pdf,16.06.2022 18:25:30,22.3067,"Mo. Knecht. Aufhebung des Verbots, Rahmenbewil...",Abstimmung über die Motion,-,Nein,Noser Ruedi,ZH,0,beide absent,---
38607,SR_5111_2109.pdf,30.09.2021 11:21:42,21.005-1,Internationale Arbeitsorganisation. Übereinkom...,Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---
39251,SR_5111_2109.pdf,28.09.2021 12:24:57,21.025-1,Zusatzkredit «Umfahrung Oberburg»,Gesamtabstimmung,+,Nein,Noser Ruedi,ZH,0,beide absent,---


In [361]:
df_züri.to_excel("Zuerich_absent.xlsx")  

In [370]:
df_komplett[df_komplett["Standesstimme"].isin (["beide dafür", "beide dagegen"])]["Kanton"].value_counts()

GE    2368
VS    2306
LU    2256
UR    2208
FR    2176
ZG    2164
SH    2142
GR    2134
AG    2012
VD    1966
JU    1954
SO    1922
NE    1670
SG    1620
SZ    1490
TG    1374
TI    1352
GL    1324
ZH    1280
BE    1142
Name: Kanton, dtype: int64

In [371]:
df_komplett[df_komplett["Standesstimme"] == "uneinig"]["Kanton"].value_counts()

BE    1080
NE     946
TI     904
AG     646
SO     646
JU     640
VD     616
SH     568
SG     550
GL     526
ZH     518
SZ     442
FR     436
TG     430
ZG     412
UR     354
GR     342
VS     338
LU     314
GE     146
Name: Kanton, dtype: int64

In [373]:
df_komplett[df_komplett["Standesstimme"].isin(["nicht vollzählig", "beide absent"])]["Kanton"].value_counts()

ZH    1184
TG    1178
GL    1122
SZ    1050
BE     756
SG     696
GR     502
GE     440
UR     418
SO     414
LU     412
TI     410
ZG     400
VD     398
NE     366
JU     364
FR     356
VS     334
AG     324
SH     242
Name: Kanton, dtype: int64

In [380]:
df_einig = df_komplett[df_komplett["Standesstimme"].isin (["beide dafür", "beide dagegen"])]["Kanton"].value_counts().to_frame()

In [ ]:
# hier rechne ich die Einigkeit hoch hypothetische 100% Anwesenheit:

In [394]:
df_abwesenheit = df_komplett[df_komplett["Standesstimme"].isin(["nicht vollzählig", "beide absent"])]["Kanton"].value_counts().to_frame()
df_abwesenheit["Prozentsatz"] = 100 / 1491 * (df_abwesenheit["Kanton"] / 2)
df_abwesenheit = df_abwesenheit.rename(columns = {"Kanton": "nv"})
df_abwesenheit = pd.concat([df_abwesenheit, df_einig], axis=1)
df_abwesenheit = df_abwesenheit.rename(columns = {"Kanton": "einig"})
df_abwesenheit["einig_rel"] = (df_abwesenheit["einig"]/2/(100-df_abwesenheit["Prozentsatz"])*100).round(0)
df_abwesenheit.sort_values("einig_rel")

,nv,Prozentsatz,einig,einig_rel
BE,756,25.352113,1142,765.0
TI,410,13.749162,1352,784.0
NE,366,12.273642,1670,952.0
SG,696,23.340040,1620,1057.0
ZH,1184,39.704896,1280,1061.0
GL,1122,37.625755,1324,1061.0
JU,364,12.206573,1954,1113.0
SO,414,13.883300,1922,1116.0
AG,324,10.865191,2012,1129.0
VD,398,13.346747,1966,1134.0


In [395]:
df_abwesenheit["einig_rel"].mean()

1133.6